In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import numpy as np
import torch
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import time
from torch.utils.data import DataLoader
import pandas as pd

In [3]:
cuda = torch.cuda.is_available()
cuda

True

In [4]:
path_name = '/content/gdrive/MyDrive/11785/idl-fall2021-hw1p2'
#train_data = np.load(os.path.join(path_name,'train.npy'),allow_pickle=True)

In [ ]:
train_data = np.load(os.path.join(path_name,'train.npy'),allow_pickle=True)


In [ ]:
#train_labels[0].shape

(1184,)

In [ ]:
train_data[0].shape[0]

1184

In [5]:
class MySpeechDataset(torch.utils.data.Dataset):
  def __init__(self,data,k):
    self.x = data[0]
    self.y = data[1]
    self.k = k
    index_map_x = []
    for i,x_data in enumerate(self.x):
      for j in range(x_data.shape[0]):
        index_pairs = (i,j)
        index_map_x.append(index_pairs)
    self.index_map = index_map_x    
    self.length = len(self.index_map)

  def __len__(self):
    return len(self.index_map)

  def __getitem__(self, index):
    i, j = self.index_map[index]
    ylabel = self.y[i][j] 
    x_with_context = self.x[i].take(range(j-self.k,j+self.k+1),axis=0,mode='clip').reshape(-1)
    xfeature = torch.from_numpy(x_with_context).float()
    return xfeature, ylabel

In [6]:
class MySpeechDatasettest(torch.utils.data.Dataset):
  def __init__(self,data,k):
    self.x = data
    self.k = k
    index_map_x = []
    for i,x_data in enumerate(self.x):
      for j in range(x_data.shape[0]):
        index_pairs = (i,j)
        index_map_x.append(index_pairs)
    self.index_map = index_map_x    
    self.length = len(self.index_map)

  def __len__(self):
    return len(self.index_map)

  def __getitem__(self, index):
    i, j = self.index_map[index]
    x_with_context = self.x[i].take(range(j-self.k,j+self.k+1),axis=0,mode='clip').reshape(-1)
    xfeature = torch.from_numpy(x_with_context).float()
    return xfeature

In [7]:
num_workers = 4 if cuda else 0 

In [8]:
#training
data_train = (np.load(os.path.join(path_name,'train.npy'),allow_pickle=True), np.load(os.path.join(path_name,'train_labels.npy'),allow_pickle=True))
train_dataset = MySpeechDataset(data_train, k=20)
train_loader_args = dict(shuffle=True, batch_size=256, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=32)
train_loader = DataLoader(train_dataset, **train_loader_args)

#data_validation = (np.load(os.path.join(path_name,'dev.npy'),allow_pickle=True), np.load(os.path.join(path_name,'dev_labels.npy'),allow_pickle=True))
#validation_dataset = MySpeechDataset(data_validation, k=20)
#validation_loader_args = dict(shuffle=False, batch_size=32, num_workers=num_workers)
#validation_loader = DataLoader(validation_dataset, **validation_loader_args)

data_test = np.load(os.path.join(path_name,'test.npy'),allow_pickle=True)
test_dataset = MySpeechDatasettest(data_test, k=20)
test_loader_args = dict(shuffle=False, batch_size=32, num_workers=num_workers)
test_loader = DataLoader(test_dataset, **test_loader_args)


In [9]:
data_validation = (np.load(os.path.join(path_name,'dev.npy'),allow_pickle=True), np.load(os.path.join(path_name,'dev_labels.npy'),allow_pickle=True))
validation_dataset = MySpeechDataset(data_validation, k=20)
validation_loader_args = dict(shuffle=False, batch_size=32, num_workers=num_workers)
validation_loader = DataLoader(validation_dataset, **validation_loader_args)


In [10]:
class SpeechMLP(nn.Module):
  def __init__(self):
    super(SpeechMLP, self).__init__()
    self.model = nn.Sequential(nn.Linear(1640,2048), nn.BatchNorm1d(2048), nn.LeakyReLU(), nn.Linear(2048,1024), nn.BatchNorm1d(1024), nn.LeakyReLU(), nn.Linear(1024,1024), nn.BatchNorm1d(1024), nn.LeakyReLU(), nn.Linear(1024,1024), nn.BatchNorm1d(1024), nn.LeakyReLU(), nn.Linear(1024,1024), nn.BatchNorm1d(1024), nn.LeakyReLU(), nn.Linear(1024,1024), nn.BatchNorm1d(1024), nn.LeakyReLU(), nn.Linear(1024,512),nn.LeakyReLU(), nn.Linear(512,256),nn.LeakyReLU(), nn.Linear(256,71))
  def forward(self,x):
    return self.model(x)


In [11]:
model = SpeechMLP()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)
print(model)

SpeechMLP(
  (model): Sequential(
    (0): Linear(in_features=1640, out_features=2048, bias=True)
    (1): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=1024, out_features=1024, bias=True)
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Linear(in_features=1024, out_features=1024, bias=True)
    (10): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Linear(in_features=1024, out_features=1024, bias=True)
    (13): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): LeakyReLU(negativ

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):   
        optimizer.zero_grad()   
        data = data.to(device)
        target = target.type(torch.LongTensor)
        target = target.to(device) 
        
        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()
    
    end_time = time.time()
    
    running_loss /= len(train_loader)
    print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss

In [12]:
def validation_model(model, validation_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(validation_loader):   
            data = data.to(device)
            target = target.type(torch.LongTensor)
            target = target.to(device)

            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()


        running_loss /= len(validation_loader)
        acc = (correct_predictions/total_predictions)*100.0
        print('Validation Loss: ', running_loss)
        print('Validation Accuracy: ', acc, '%')
        return running_loss, acc


In [16]:
def test_model(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        predictions = []
        for batch_idx, (data) in enumerate(test_loader):   
            data = data.to(device)
            #target = target.type(torch.LongTensor)
            #target = target.to(device)
            
            outputs = model(data)

            _, predicted = torch.max(outputs.data, 1)
            predictions.append(predicted)
            
        
        predictions = torch.cat(predictions, axis=0).cpu().numpy()
        #predictions = np.array(list(predictions))
        num_predictions = np.arange(0,len(predictions))
        df = pd.DataFrame()
        df['id'] = num_predictions
        df['label'] = predictions
        df.to_csv(os.path.join(path_name,'predictions.csv'),index=False)

        
        #running_loss /= len(test_loader)
        #acc = (correct_predictions/total_predictions)*100.0
        #print('Testing Loss: ', running_loss)
        #print('Testing Accuracy: ', acc, '%')
        #return running_loss, acc

In [ ]:
n_epochs = 5
Train_loss = []
#Validation_loss = []
#Validation_acc = []
Test_loss = []
Test_acc = []

for i in range(n_epochs):
    train_loss = train_epoch(model, train_loader, criterion, optimizer)
    #validation_loss, validation_acc = validation_model(model, validation_loader, criterion)
    #test_loss, test_acc = test_model(model, test_loader, criterion)
    Train_loss.append(train_loss)
    #Validation_loss.append(validation_loss)
    #Validation_acc.append(validation_acc)
    #Test_loss.append(test_loss)
    #Test_acc.append(test_acc)
    print('='*20)
    model_path = os.path.join(path_name, 'model.pt')
    torch.save(model, model_path)

Training Loss:  0.8637536383286077 Time:  2348.9291043281555 s
Training Loss:  0.5987557614687572 Time:  2347.7928614616394 s
Training Loss:  0.5078717617570858 Time:  2355.4739983081818 s
Training Loss:  0.45006239256163716 Time:  2349.14657497406 s
Training Loss:  0.407503372262183 Time:  2345.305848121643 s


In [ ]:
validation_loss, validation_acc = validation_model(model, validation_loader, criterion)

Validation Loss:  0.9393573400696338
Validation Accuracy:  75.22427646462283 %


In [14]:
model = torch.load(os.path.join(path_name, 'model.pt'))

In [ ]:
model_path = os.path.join(path_name, 'model.pt')
torch.save(model, model_path)

In [ ]:
test_model(model, test_loader, criterion)